In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

# Load relevant tables

In [2]:
# VCT 2021
matches_overview_2021 = pd.read_csv("vct_data/vct_2021/matches/overview.csv")
matches_maps_scores_2021 = pd.read_csv("vct_data/vct_2021/matches/maps_scores.csv")
matches_eco_rounds_2021 = pd.read_csv("vct_data/vct_2021/matches/eco_rounds.csv")
matches_kills_stats_2021 = pd.read_csv("vct_data/vct_2021/matches/kills_stats.csv")
matches_win_loss_methods_count_2021 = pd.read_csv("vct_data/vct_2021/matches/win_loss_methods_count.csv")

# VCT 2022
matches_overview_2022 = pd.read_csv("vct_data/vct_2022/matches/overview.csv")
matches_maps_scores_2022 = pd.read_csv("vct_data/vct_2022/matches/maps_scores.csv")
matches_eco_rounds_2022 = pd.read_csv("vct_data/vct_2022/matches/eco_rounds.csv")
matches_kills_stats_2022 = pd.read_csv("vct_data/vct_2022/matches/kills_stats.csv")
matches_win_loss_methods_count_2022 = pd.read_csv("vct_data/vct_2022/matches/win_loss_methods_count.csv")

# VCT 2023
matches_overview_2023 = pd.read_csv("vct_data/vct_2023/matches/overview.csv")
matches_maps_scores_2023 = pd.read_csv("vct_data/vct_2023/matches/maps_scores.csv")
matches_eco_rounds_2023 = pd.read_csv("vct_data/vct_2023/matches/eco_rounds.csv")
matches_kills_stats_2023 = pd.read_csv("vct_data/vct_2023/matches/kills_stats.csv")
matches_win_loss_methods_count_2023 = pd.read_csv("vct_data/vct_2023/matches/win_loss_methods_count.csv")

# VCT 2024
matches_overview_2024 = pd.read_csv("vct_data/vct_2024/matches/overview.csv")
matches_maps_scores_2024 = pd.read_csv("vct_data/vct_2024/matches/maps_scores.csv")
matches_eco_rounds_2024 = pd.read_csv("vct_data/vct_2024/matches/eco_rounds.csv")
matches_kills_stats_2024 = pd.read_csv("vct_data/vct_2024/matches/kills_stats.csv")
matches_win_loss_methods_count_2024 = pd.read_csv("vct_data/vct_2024/matches/win_loss_methods_count.csv")

C:\Users\Red\AppData\Local\Temp\ipykernel_7832\2025941948.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_overview_2021 = pd.read_csv("vct_data/vct_2021/matches/overview.csv")


# Merge tables by columns (per year)

In [3]:
def merge_tables(matches_maps_scores, matches_overview, matches_eco_rounds, matches_kills_stats, matches_win_loss_methods_count):
    # Base table
    vct_data = matches_maps_scores.copy()

    # Tournament is sorted in descending order, reverse it to ascending order (oldest to newest)
    tourna_group = vct_data.groupby("Tournament", sort=False)
    reversed_tourna = list(tourna_group.groups.keys())[::-1]
    vct_data = pd.concat([tourna_group.get_group(tourna) for tourna in reversed_tourna]).reset_index(drop=True)

    # Aggregate and merge overview stats
    matches_overview_both = matches_overview[matches_overview["Side"] == "both"]

    overview_agg = matches_overview_both.groupby(["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"]).agg({
        "Rating": "mean",
        "Average Combat Score": "mean",
        "Kills": "sum",
        "Deaths": "sum",
        "Assists": "sum",
        "Kills - Deaths (KD)": "sum",
        "Kill, Assist, Trade, Survive %": lambda x: np.mean([(float(i.strip("%")) / 100) if isinstance(i, str) else i for i in x]),
        "Average Damage Per Round": "mean",
        "Headshot %": lambda x: np.mean([(float(i.strip("%")) / 100) if isinstance(i, str) else i for i in x]),
        "First Kills": "sum",
        "First Deaths": "sum",
        "Kills - Deaths (FKD)": "sum"
    }).reset_index()

    vct_data = vct_data.merge(
        overview_agg, 
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team A"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left"
    )

    vct_data = vct_data.merge(
        overview_agg, 
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team B"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left",
        suffixes=("_TeamA", "_TeamB")
    )

    # Always drop these columns after merging for Team B to avoid duplicate col error
    vct_data = vct_data.drop(columns=["Team_TeamA", "Team_TeamB"])

    # Aggregate and merge for eco rounds
    eco_rounds_agg = matches_eco_rounds.groupby(["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"]).agg({
        "Loadout Value": lambda x: np.mean([float(i.replace('k', '').replace(',', '')) * 1000 for i in x]),
        "Remaining Credits": lambda x: np.mean([float(i.replace('k', '').replace(',', '')) * 1000 for i in x]),
        "Type": lambda x: x.mode()[0]
    }).reset_index()

    vct_data = vct_data.merge(
        eco_rounds_agg,
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team A"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left"
    )

    vct_data = vct_data.merge(
        eco_rounds_agg,
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team B"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left",
        suffixes=("_TeamA", "_TeamB")
    )

    vct_data = vct_data.drop(columns=["Team_TeamA", "Team_TeamB"])

    # Aggregate and merge for kills stats
    kills_stats_agg = matches_kills_stats.groupby(["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"]).agg({
        "2k": "sum",
        "3k": "sum",
        "4k": "sum",
        "5k": "sum",
        "1v1": "sum",
        "1v2": "sum",
        "1v3": "sum",
        "1v4": "sum",
        "1v5": "sum",
        "Econ": "mean",
        "Spike Plants": "sum",
        "Spike Defuses": "sum"
    }).reset_index()

    vct_data = vct_data.merge(
        kills_stats_agg,
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team A"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left"
    )

    vct_data = vct_data.merge(
        kills_stats_agg,
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team B"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left",
        suffixes=("_TeamA", "_TeamB")
    )

    vct_data = vct_data.drop(columns=["Team_TeamA", "Team_TeamB"])
    
    # Merge with win/loss methods count
    vct_data = vct_data.merge(
        matches_win_loss_methods_count,
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team A"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left"
    )

    vct_data = vct_data.merge(
        matches_win_loss_methods_count,
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team B"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left",
        suffixes=("_TeamA", "_TeamB")
    )

    vct_data = vct_data.drop(columns=["Team_TeamA", "Team_TeamB"])

    # Add prediction target
    vct_data["Winner"] = np.where(vct_data["Team A Score"] > vct_data["Team B Score"], 1, 0)

    # Remove duplicates
    vct_data.drop_duplicates(inplace=True)

    return vct_data


# Merge tables for each year
vct_2021 = merge_tables(
    matches_maps_scores_2021, 
    matches_overview_2021, 
    matches_eco_rounds_2021, 
    matches_kills_stats_2021, 
    matches_win_loss_methods_count_2021,
)

vct_2022 = merge_tables(
    matches_maps_scores_2022, 
    matches_overview_2022, 
    matches_eco_rounds_2022, 
    matches_kills_stats_2022, 
    matches_win_loss_methods_count_2022,
)

vct_2023 = merge_tables(
    matches_maps_scores_2023, 
    matches_overview_2023, 
    matches_eco_rounds_2023, 
    matches_kills_stats_2023, 
    matches_win_loss_methods_count_2023,
)

vct_2024 = merge_tables(
    matches_maps_scores_2024, 
    matches_overview_2024, 
    matches_eco_rounds_2024, 
    matches_kills_stats_2024, 
    matches_win_loss_methods_count_2024,
)

# Investigate each year's VCT data

In [4]:
vct_2021.tail()

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team A Score,Team A Attacker Score,Team A Defender Score,Team A Overtime Score,Team B,Team B Score,Team B Attacker Score,Team B Defender Score,Team B Overtime Score,Duration,Rating_TeamA,Average Combat Score_TeamA,Kills_TeamA,Deaths_TeamA,Assists_TeamA,Kills - Deaths (KD)_TeamA,"Kill, Assist, Trade, Survive %_TeamA",Average Damage Per Round_TeamA,Headshot %_TeamA,First Kills_TeamA,First Deaths_TeamA,Kills - Deaths (FKD)_TeamA,Rating_TeamB,Average Combat Score_TeamB,Kills_TeamB,Deaths_TeamB,Assists_TeamB,Kills - Deaths (KD)_TeamB,"Kill, Assist, Trade, Survive %_TeamB",Average Damage Per Round_TeamB,Headshot %_TeamB,First Kills_TeamB,First Deaths_TeamB,Kills - Deaths (FKD)_TeamB,Loadout Value_TeamA,Remaining Credits_TeamA,Type_TeamA,Loadout Value_TeamB,Remaining Credits_TeamB,Type_TeamB,2k_TeamA,3k_TeamA,4k_TeamA,5k_TeamA,1v1_TeamA,1v2_TeamA,1v3_TeamA,1v4_TeamA,1v5_TeamA,Econ_TeamA,Spike Plants_TeamA,Spike Defuses_TeamA,2k_TeamB,3k_TeamB,4k_TeamB,5k_TeamB,1v1_TeamB,1v2_TeamB,1v3_TeamB,1v4_TeamB,1v5_TeamB,Econ_TeamB,Spike Plants_TeamB,Spike Defuses_TeamB,Elimination_TeamA,Detonated_TeamA,Defused_TeamA,Time Expiry (No Plant)_TeamA,Eliminated_TeamA,Defused Failed_TeamA,Detonation Denied_TeamA,Time Expiry (Failed to Plant)_TeamA,Elimination_TeamB,Detonated_TeamB,Defused_TeamB,Time Expiry (No Plant)_TeamB,Eliminated_TeamB,Defused Failed_TeamB,Detonation Denied_TeamB,Time Expiry (Failed to Plant)_TeamB,Winner
14630,Valorant Champions 2021,Playoffs,Grand Final,Acend vs Gambit Esports,Breeze,Acend,11,9,2,NaN,Gambit Esports,13,10,3.0,NaN,1:12:20,0.818,156.0,66.0,88.0,19.0,-22.0,0.568,104.2,0.396,12.0,12.0,0.0,1.156,207.6,88.0,66.0,21.0,22.0,0.726,140.8,0.248,12.0,12.0,0.0,18200.000000,8575.000000,Full buy: 20k+,19437.500000,10587.500000,Full buy: 20k+,11.0,2.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,41.2,2.0,4.0,17.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0,7.0,0.0,4.0,1.0,4.0,2.0,13.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,4.0,1.0,4.0,2.0,0
14631,Valorant Champions 2021,Playoffs,Grand Final,Acend vs Gambit Esports,Ascent,Acend,13,7,6,NaN,Gambit Esports,7,2,5.0,NaN,43:25,1.126,202.8,75.0,56.0,35.0,19.0,0.770,136.4,0.312,10.0,10.0,0.0,0.900,168.4,56.0,75.0,22.0,-19.0,0.630,112.0,0.364,10.0,10.0,0.0,19525.000000,13455.000000,Full buy: 20k+,15530.000000,4035.000000,Semi-buy: 10-20k,15.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,7.0,1.0,10.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,47.0,3.0,0.0,10.0,1.0,1.0,1.0,6.0,1.0,0.0,0.0,6.0,1.0,0.0,0.0,10.0,1.0,1.0,1.0,1
14632,Valorant Champions 2021,Playoffs,Grand Final,Acend vs Gambit Esports,Fracture,Acend,3,1,2,NaN,Gambit Esports,13,2,11.0,NaN,33:16,0.444,120.0,30.0,75.0,8.0,-45.0,0.412,78.4,0.216,4.0,12.0,-8.0,1.498,239.0,74.0,30.0,25.0,44.0,0.952,157.4,0.304,12.0,4.0,8.0,13725.000000,3825.000000,Full buy: 20k+,18418.750000,18237.500000,Full buy: 20k+,6.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,31.8,2.0,1.0,12.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,92.4,9.0,1.0,2.0,0.0,1.0,0.0,12.0,0.0,1.0,0.0,12.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,0
14633,Valorant Champions 2021,Playoffs,Grand Final,Acend vs Gambit Esports,Icebox,Acend,14,7,5,2.0,Gambit Esports,12,7,5.0,0.0,53:21,1.026,205.2,93.0,95.0,36.0,-2.0,0.746,136.4,0.270,11.0,15.0,-4.0,0.982,209.4,95.0,93.0,28.0,2.0,0.730,141.6,0.302,15.0,11.0,4.0,18703.846154,7561.538462,Full buy: 20k+,19688.461538,6276.923077,Full buy: 20k+,19.0,4.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,49.4,7.0,3.0,23.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,51.8,8.0,1.0,10.0,1.0,3.0,0.0,11.0,0.0,1.0,0.0,11.0,0.0,1.0,0.0,10.0,1.0,3.0,0.0,1
14634,Valorant Champions 2021,Playoffs,Grand Final,Acend vs Gambit Esports,Split,Acend,13,7,6,NaN,Gambit Esports,8,3,5.0,NaN,46:13,1.088,210.6,82.0,63.0,28.0,19.0,0.740,138.2,0.270,13.0,8.0,5.0,0.878,177.4,63.0,82.0,17.0,-19.0,0.562,122.4,0.214,8.0,13.0,-5.0,19090.476190,8933.333333,Full buy: 20k+,15457.142857,5042.857143,Semi-buy: 10-20k,16.0,3.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,54.0,6.0,2.0,13.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,50.0,7.0,0.0,9.0,2.0,2.0,0.0,7.0,1.0,0.0,0.0,7.0,1

In [5]:
vct_2022.tail()

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team A Score,Team A Attacker Score,Team A Defender Score,Team A Overtime Score,Team B,Team B Score,Team B Attacker Score,Team B Defender Score,Team B Overtime Score,Duration,Rating_TeamA,Average Combat Score_TeamA,Kills_TeamA,Deaths_TeamA,Assists_TeamA,Kills - Deaths (KD)_TeamA,"Kill, Assist, Trade, Survive %_TeamA",Average Damage Per Round_TeamA,Headshot %_TeamA,First Kills_TeamA,First Deaths_TeamA,Kills - Deaths (FKD)_TeamA,Rating_TeamB,Average Combat Score_TeamB,Kills_TeamB,Deaths_TeamB,Assists_TeamB,Kills - Deaths (KD)_TeamB,"Kill, Assist, Trade, Survive %_TeamB",Average Damage Per Round_TeamB,Headshot %_TeamB,First Kills_TeamB,First Deaths_TeamB,Kills - Deaths (FKD)_TeamB,Loadout Value_TeamA,Remaining Credits_TeamA,Type_TeamA,Loadout Value_TeamB,Remaining Credits_TeamB,Type_TeamB,2k_TeamA,3k_TeamA,4k_TeamA,5k_TeamA,1v1_TeamA,1v2_TeamA,1v3_TeamA,1v4_TeamA,1v5_TeamA,Econ_TeamA,Spike Plants_TeamA,Spike Defuses_TeamA,2k_TeamB,3k_TeamB,4k_TeamB,5k_TeamB,1v1_TeamB,1v2_TeamB,1v3_TeamB,1v4_TeamB,1v5_TeamB,Econ_TeamB,Spike Plants_TeamB,Spike Defuses_TeamB,Elimination_TeamA,Detonated_TeamA,Defused_TeamA,Time Expiry (No Plant)_TeamA,Eliminated_TeamA,Defused Failed_TeamA,Detonation Denied_TeamA,Time Expiry (Failed to Plant)_TeamA,Elimination_TeamB,Detonated_TeamB,Defused_TeamB,Time Expiry (No Plant)_TeamB,Eliminated_TeamB,Defused Failed_TeamB,Detonation Denied_TeamB,Time Expiry (Failed to Plant)_TeamB,Winner
8903,Valorant Champions 2022,Playoffs,Lower Final,OpTic Gaming vs DRX,Haven,OpTic Gaming,13,8,5,NaN,DRX,10,6,4,NaN,51:53,1.206,221.4,89.0,68.0,38.0,21.0,0.810,142.4,0.276,12.0,11.0,1.0,0.806,173.2,68.0,89.0,18.0,-21.0,0.618,120.8,0.314,11.0,12.0,-1.0,18821.739130,7769.565217,Full buy: 20k+,18121.739130,7626.086957,Full buy: 20k+,14.0,7.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,59.4,6.0,0.0,11.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,46.2,4.0,2.0,13.0,0.0,0.0,0.0,6.0,2.0,2.0,0.0,6.0,2.0,2.0,0.0,13.0,0.0,0.0,0.0,1
8904,Valorant Champions 2022,Playoffs,Grand Final,LOUD vs OpTic Gaming,Ascent,LOUD,15,5,7,3.0,OpTic Gaming,13,5,7,1.0,1:23:44,1.154,222.2,108.0,99.0,55.0,9.0,0.778,142.2,0.246,15.0,13.0,2.0,0.882,202.8,99.0,108.0,42.0,-9.0,0.690,134.2,0.246,13.0,15.0,-2.0,18678.571429,4353.571429,Full buy: 20k+,19232.142857,6789.285714,Full buy: 20k+,19.0,8.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,47.8,6.0,3.0,17.0,5.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,45.6,8.0,1.0,12.0,0.0,3.0,0.0,11.0,1.0,1.0,0.0,11.0,1.0,1.0,0.0,12.0,0.0,3.0,0.0,1
8905,Valorant Champions 2022,Playoffs,Grand Final,LOUD vs OpTic Gaming,Bind,LOUD,6,2,4,NaN,OpTic Gaming,13,3,10,NaN,56:19,0.800,167.0,52.0,72.0,19.0,-20.0,0.642,110.6,0.234,7.0,11.0,-4.0,1.168,208.2,72.0,52.0,22.0,20.0,0.832,138.6,0.240,12.0,8.0,4.0,17400.000000,5436.842105,Full buy: 20k+,18784.210526,11368.421053,Full buy: 20k+,8.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,43.2,6.0,1.0,11.0,5.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,63.0,11.0,2.0,3.0,2.0,1.0,0.0,7.0,3.0,2.0,1.0,7.0,3.0,2.0,1.0,3.0,2.0,1.0,0.0,0
8906,Valorant Champions 2022,Playoffs,Grand Final,LOUD vs OpTic Gaming,Breeze,LOUD,16,6,6,4.0,OpTic Gaming,14,6,6,2.0,1:10:53,1.152,208.2,109.0,87.0,33.0,22.0,0.712,135.8,0.312,12.0,18.0,-6.0,0.834,169.0,87.0,109.0,22.0,-22.0,0.598,113.8,0.250,18.0,12.0,6.0,19023.333333,6663.333333,Full buy: 20k+,20016.666667,5083.333333,Full buy: 20k+,18.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,13.0,6.0,9.0,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,38.4,13.0,5.0,7.0,3.0,6.0,0.0,4.0,4.0,5.0,1.0,4.0,4.0,5.0,1.0,7.0,3.0,6.0,0.0,1
8907,Valorant Champions 2022,Playoffs,Grand Final,LOUD vs OpTic Gaming,Haven,LOUD,13,7,6,NaN,OpTic Gaming,5,0,5,NaN,41:16,1.278,226.4,71.0,47.0,42.0,24.0,0.778,142.4,0.286,7.0,11.0,-4.0,0.738,160.4,47.0,71.0,20.0,-24.0,0.558,102.2,0.246,11.0,7.0,4.0,19694.444444,10311.111111,Full buy: 20k+,16638.888889,7155.555556,Full buy: 20k+,13.0,5.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,57.6,5.0,4.0,10.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,41.4,9.0,0.0,8.0,1.0,4.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,8.0,1.0,4.0,0.0,1


In [6]:
vct_2023.head()

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team A Score,Team A Attacker Score,Team A Defender Score,Team A Overtime Score,Team B,Team B Score,Team B Attacker Score,Team B Defender Score,Team B Overtime Score,Duration,Rating_TeamA,Average Combat Score_TeamA,Kills_TeamA,Deaths_TeamA,Assists_TeamA,Kills - Deaths (KD)_TeamA,"Kill, Assist, Trade, Survive %_TeamA",Average Damage Per Round_TeamA,Headshot %_TeamA,First Kills_TeamA,First Deaths_TeamA,Kills - Deaths (FKD)_TeamA,Rating_TeamB,Average Combat Score_TeamB,Kills_TeamB,Deaths_TeamB,Assists_TeamB,Kills - Deaths (KD)_TeamB,"Kill, Assist, Trade, Survive %_TeamB",Average Damage Per Round_TeamB,Headshot %_TeamB,First Kills_TeamB,First Deaths_TeamB,Kills - Deaths (FKD)_TeamB,Loadout Value_TeamA,Remaining Credits_TeamA,Type_TeamA,Loadout Value_TeamB,Remaining Credits_TeamB,Type_TeamB,2k_TeamA,3k_TeamA,4k_TeamA,5k_TeamA,1v1_TeamA,1v2_TeamA,1v3_TeamA,1v4_TeamA,1v5_TeamA,Econ_TeamA,Spike Plants_TeamA,Spike Defuses_TeamA,2k_TeamB,3k_TeamB,4k_TeamB,5k_TeamB,1v1_TeamB,1v2_TeamB,1v3_TeamB,1v4_TeamB,1v5_TeamB,Econ_TeamB,Spike Plants_TeamB,Spike Defuses_TeamB,Elimination_TeamA,Detonated_TeamA,Defused_TeamA,Time Expiry (No Plant)_TeamA,Eliminated_TeamA,Defused Failed_TeamA,Detonation Denied_TeamA,Time Expiry (Failed to Plant)_TeamA,Elimination_TeamB,Detonated_TeamB,Defused_TeamB,Time Expiry (No Plant)_TeamB,Eliminated_TeamB,Defused Failed_TeamB,Detonation Denied_TeamB,Time Expiry (Failed to Plant)_TeamB,Winner
0,Champions Tour 2023: Lock-In Sao Paulo,Bracket Stage,Alpha - Round of 16,KOI vs NRG Esports,Icebox,KOI,11,9,2,NaN,NRG Esports,13,10,3,NaN,1:12:59,0.904,174.4,73.0,83.0,36.0,-10.0,0.642,118.0,0.260,8.0,16.0,-8.0,1.126,204.2,83.0,73.0,31.0,10.0,0.750,131.4,0.218,16.0,8.0,8.0,18391.666667,12425.000000,Full buy: 20k+,18045.833333,12916.666667,Full buy: 20k+,16.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,47.8,9.0,1.0,13.0,5.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,57.6,10.0,1.0,9,1,1,0,9,3,1,0,9,3,1,0,9,1,1,0,0
1,Champions Tour 2023: Lock-In Sao Paulo,Bracket Stage,Alpha - Round of 16,KOI vs NRG Esports,Haven,KOI,9,4,5,NaN,NRG Esports,13,5,8,NaN,53:43,0.866,166.6,62.0,69.0,27.0,-7.0,0.720,107.2,0.238,8.0,14.0,-6.0,1.146,188.2,69.0,62.0,38.0,7.0,0.788,121.8,0.278,14.0,8.0,6.0,18195.454545,5986.363636,Full buy: 20k+,18659.090909,9200.000000,Full buy: 20k+,8.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0,9.0,3.0,13.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,51.0,11.0,4.0,4,2,3,0,4,5,4,0,4,5,4,0,4,2,3,0,0
2,Champions Tour 2023: Lock-In Sao Paulo,Bracket Stage,Alpha - Round of 16,DetonatioN FocusMe vs Giants Gaming,Haven,DetonatioN FocusMe,4,4,0,NaN,Giants Gaming,13,5,8,NaN,50:48,0.574,154.4,45.0,71.0,16.0,-26.0,0.612,102.8,0.342,7.0,10.0,-3.0,1.428,225.8,71.0,45.0,31.0,26.0,0.904,149.0,0.260,10.0,7.0,3.0,15847.058824,6188.235294,Full buy: 20k+,18100.000000,8894.117647,Full buy: 20k+,7.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,42.0,8.0,0.0,14.0,4.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,66.6,5.0,4.0,3,1,0,0,7,2,4,0,7,2,4,0,3,1,0,0,0
3,Champions Tour 2023: Lock-In Sao Paulo,Bracket Stage,Alpha - Round of 16,DetonatioN FocusMe vs Giants Gaming,Icebox,DetonatioN FocusMe,7,4,3,NaN,Giants Gaming,13,5,8,NaN,51:43,0.830,179.6,59.0,78.0,25.0,-19.0,0.600,114.8,0.316,8.0,12.0,-4.0,1.184,209.8,78.0,59.0,24.0,19.0,0.780,148.6,0.354,12.0,8.0,4.0,16225.000000,5620.000000,Full buy: 20k+,19410.000000,12570.000000,Full buy: 20k+,14.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,47.6,4.0,2.0,20.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,62.8,10.0,1.0,4,0,2,1,11,0,1,1,11,0,1,1,4,0,2,1,0
4,Champions Tour 2023: Lock-In Sao Paulo,Bracket Stage,Alpha - Round of 16,FunPlus Phoenix vs Karmine Corp,Lotus,FunPlus Phoenix,6,4,2,NaN,Karmine Corp,13,5,8,NaN,55:34,0.782,176.2,56.0,79.0,30.0,-23.0,0.684,111.8,0.250,2.0,17.0,-15.0,1.220,225.2,79.0,56.0,33.0,23.0,0.768,140.4,0.328,17.0,2.0,15.0,15978.947368,5631.578947,Full buy: 20k+,18100.000000,7052.631579,Full buy: 20k+,7.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,46.6,6.0,0.0,13.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,57.0,5.0,2.0,4,1,0,1,11,0,2,0,11,0,2,0,4,

In [7]:
vct_2024.head()

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team A Score,Team A Attacker Score,Team A Defender Score,Team A Overtime Score,Team B,Team B Score,Team B Attacker Score,Team B Defender Score,Team B Overtime Score,Duration,Rating_TeamA,Average Combat Score_TeamA,Kills_TeamA,Deaths_TeamA,Assists_TeamA,Kills - Deaths (KD)_TeamA,"Kill, Assist, Trade, Survive %_TeamA",Average Damage Per Round_TeamA,Headshot %_TeamA,First Kills_TeamA,First Deaths_TeamA,Kills - Deaths (FKD)_TeamA,Rating_TeamB,Average Combat Score_TeamB,Kills_TeamB,Deaths_TeamB,Assists_TeamB,Kills - Deaths (KD)_TeamB,"Kill, Assist, Trade, Survive %_TeamB",Average Damage Per Round_TeamB,Headshot %_TeamB,First Kills_TeamB,First Deaths_TeamB,Kills - Deaths (FKD)_TeamB,Loadout Value_TeamA,Remaining Credits_TeamA,Type_TeamA,Loadout Value_TeamB,Remaining Credits_TeamB,Type_TeamB,2k_TeamA,3k_TeamA,4k_TeamA,5k_TeamA,1v1_TeamA,1v2_TeamA,1v3_TeamA,1v4_TeamA,1v5_TeamA,Econ_TeamA,Spike Plants_TeamA,Spike Defuses_TeamA,2k_TeamB,3k_TeamB,4k_TeamB,5k_TeamB,1v1_TeamB,1v2_TeamB,1v3_TeamB,1v4_TeamB,1v5_TeamB,Econ_TeamB,Spike Plants_TeamB,Spike Defuses_TeamB,Elimination_TeamA,Detonated_TeamA,Defused_TeamA,Time Expiry (No Plant)_TeamA,Eliminated_TeamA,Defused Failed_TeamA,Detonation Denied_TeamA,Time Expiry (Failed to Plant)_TeamA,Elimination_TeamB,Detonated_TeamB,Defused_TeamB,Time Expiry (No Plant)_TeamB,Eliminated_TeamB,Defused Failed_TeamB,Detonation Denied_TeamB,Time Expiry (Failed to Plant)_TeamB,Winner
0,Champions Tour 2024: Pacific Kickoff,Group Stage,Opening (A),T1 vs BLEED,Breeze,T1,18,6,6,6.0,BLEED,16,6,6,4.0,3:22:31,1.104,214.6,126.0,116.0,36.0,10.0,0.724,145.2,0.292,15.0,19.0,-4.0,0.850,192.6,116.0,126.0,40.0,-10.0,0.730,125.6,0.266,19.0,15.0,4.0,19458.823529,4994.117647,Full buy: 20k+,20517.647059,5367.647059,Full buy: 20k+,28.0,4.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,45.8,14.0,6.0,22.0,5.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0,37.4,13.0,6.0,11,1,6,0,7,3,6,0,7,3,6,0,11,1,6,0,1
1,Champions Tour 2024: Pacific Kickoff,Group Stage,Opening (A),T1 vs BLEED,Lotus,T1,13,9,4,NaN,BLEED,6,3,3,NaN,47:45,1.160,221.2,75.0,59.0,36.0,16.0,0.758,143.4,0.248,11.0,8.0,3.0,0.860,191.8,59.0,75.0,24.0,-16.0,0.642,128.8,0.244,8.0,11.0,-3.0,19810.526316,9421.052632,Full buy: 20k+,16073.684211,4147.368421,Full buy: 20k+,15.0,4.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,57.6,10.0,4.0,12.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,51.2,7.0,2.0,6,3,4,0,3,1,2,0,3,1,2,0,6,3,4,0,1
2,Champions Tour 2024: Pacific Kickoff,Group Stage,Opening (C),Gen.G vs Rex Regum Qeon,Icebox,Gen.G,13,6,7,NaN,Rex Regum Qeon,11,5,6,NaN,50:50,1.036,205.0,87.0,82.0,28.0,5.0,0.742,136.2,0.290,15.0,9.0,6.0,0.944,193.4,82.0,87.0,26.0,-5.0,0.692,124.4,0.320,9.0,15.0,-6.0,17695.833333,10762.500000,Full buy: 20k+,18137.500000,9095.833333,Full buy: 20k+,12.0,5.0,2.0,0.0,2.0,1.0,0.0,0.0,0.0,56.6,8.0,6.0,18.0,6.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,48.0,11.0,2.0,6,1,6,0,9,0,2,0,9,0,2,0,6,1,6,0,1
3,Champions Tour 2024: Pacific Kickoff,Group Stage,Opening (C),Gen.G vs Rex Regum Qeon,Split,Gen.G,11,2,9,NaN,Rex Regum Qeon,13,3,10,NaN,48:33,1.026,213.4,89.0,86.0,43.0,3.0,0.758,136.8,0.258,15.0,9.0,6.0,1.006,202.8,86.0,89.0,32.0,-3.0,0.692,134.8,0.312,9.0,15.0,-6.0,18575.000000,10908.333333,Full buy: 20k+,17095.833333,8683.333333,Full buy: 20k+,16.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,51.8,6.0,0.0,13.0,5.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,51.0,3.0,4.0,11,0,0,0,9,0,4,0,9,0,4,0,11,0,0,0,0
4,Champions Tour 2024: Pacific Kickoff,Group Stage,Opening (C),Gen.G vs Rex Regum Qeon,Sunset,Gen.G,13,8,5,NaN,Rex Regum Qeon,8,4,4,NaN,45:26,1.042,217.0,82.0,76.0,38.0,6.0,0.752,144.2,0.248,13.0,8.0,5.0,0.946,202.0,76.0,82.0,31.0,-6.0,0.712,123.6,0.306,8.0,13.0,-5.0,17647.619048,5690.476190,Full buy: 20k+,16347.619048,6752.380952,Semi-buy: 10-20k,16.0,4.0,2.0,0.0,3.0,1.0,1.0,0.0,0.0,55.4,6.0,6.0,9.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,49.0,10.0,1.0,5,2,6,0,6,0,1,1,6,0,1,1,5,2,6,0,1


# Flip Team A and Team B stats 

In [8]:
def flip_teams(df):
    df_flipped = df.copy()
    
    # Flip the columns for Team A and Team B
    df_flipped = df_flipped.rename(columns={
        'Team A': 'Team B',
        'Team A Score': 'Team B Score',
        'Team A Attacker Score': 'Team B Attacker Score',
        'Team A Defender Score': 'Team B Defender Score',
        'Team A Overtime Score': 'Team B Overtime Score',
        'Team B': 'Team A',
        'Team B Score': 'Team A Score',
        'Team B Attacker Score': 'Team A Attacker Score',
        'Team B Defender Score': 'Team A Defender Score',
        'Team B Overtime Score': 'Team A Overtime Score',
        'Rating_TeamA': 'Rating_TeamB',
        'Average Combat Score_TeamA': 'Average Combat Score_TeamB',
        'Kills_TeamA': 'Kills_TeamB',
        'Deaths_TeamA': 'Deaths_TeamB',
        'Assists_TeamA': 'Assists_TeamB',
        'Kills - Deaths (KD)_TeamA': 'Kills - Deaths (KD)_TeamB',
        'Kill, Assist, Trade, Survive %_TeamA': 'Kill, Assist, Trade, Survive %_TeamB',
        'Average Damage Per Round_TeamA': 'Average Damage Per Round_TeamB',
        'Headshot %_TeamA': 'Headshot %_TeamB',
        'First Kills_TeamA': 'First Kills_TeamB',
        'First Deaths_TeamA': 'First Deaths_TeamB',
        'Kills - Deaths (FKD)_TeamA': 'Kills - Deaths (FKD)_TeamB',
        'Rating_TeamB': 'Rating_TeamA',
        'Average Combat Score_TeamB': 'Average Combat Score_TeamA',
        'Kills_TeamB': 'Kills_TeamA',
        'Deaths_TeamB': 'Deaths_TeamA',
        'Assists_TeamB': 'Assists_TeamA',
        'Kills - Deaths (KD)_TeamB': 'Kills - Deaths (KD)_TeamA',
        'Kill, Assist, Trade, Survive %_TeamB': 'Kill, Assist, Trade, Survive %_TeamA',
        'Average Damage Per Round_TeamB': 'Average Damage Per Round_TeamA',
        'Headshot %_TeamB': 'Headshot %_TeamA',
        'First Kills_TeamB': 'First Kills_TeamA',
        'First Deaths_TeamB': 'First Deaths_TeamA',
        'Kills - Deaths (FKD)_TeamB': 'Kills - Deaths (FKD)_TeamA',
        'Loadout Value_TeamA': 'Loadout Value_TeamB',
        'Remaining Credits_TeamA': 'Remaining Credits_TeamB',
        'Type_TeamA': 'Type_TeamB',
        'Loadout Value_TeamB': 'Loadout Value_TeamA',
        'Remaining Credits_TeamB': 'Remaining Credits_TeamA',
        'Type_TeamB': 'Type_TeamA',
        '2k_TeamA': '2k_TeamB',
        '3k_TeamA': '3k_TeamB',
        '4k_TeamA': '4k_TeamB',
        '5k_TeamA': '5k_TeamB',
        '1v1_TeamA': '1v1_TeamB',
        '1v2_TeamA': '1v2_TeamB',
        '1v3_TeamA': '1v3_TeamB',
        '1v4_TeamA': '1v4_TeamB',
        '1v5_TeamA': '1v5_TeamB',
        'Econ_TeamA': 'Econ_TeamB',
        'Spike Plants_TeamA': 'Spike Plants_TeamB',
        'Spike Defuses_TeamA': 'Spike Defuses_TeamB',
        '2k_TeamB': '2k_TeamA',
        '3k_TeamB': '3k_TeamA',
        '4k_TeamB': '4k_TeamA',
        '5k_TeamB': '5k_TeamA',
        '1v1_TeamB': '1v1_TeamA',
        '1v2_TeamB': '1v2_TeamA',
        '1v3_TeamB': '1v3_TeamA',
        '1v4_TeamB': '1v4_TeamA',
        '1v5_TeamB': '1v5_TeamA',
        'Econ_TeamB': 'Econ_TeamA',
        'Spike Plants_TeamB': 'Spike Plants_TeamA',
        'Spike Defuses_TeamB': 'Spike Defuses_TeamA',
        'Elimination_TeamA': 'Elimination_TeamB',
        'Detonated_TeamA': 'Detonated_TeamB',
        'Defused_TeamA': 'Defused_TeamB',
        'Time Expiry (No Plant)_TeamA': 'Time Expiry (No Plant)_TeamB',
        'Eliminated_TeamA': 'Eliminated_TeamB',
        'Defused Failed_TeamA': 'Defused Failed_TeamB',
        'Detonation Denied_TeamA': 'Detonation Denied_TeamB',
        'Time Expiry (Failed to Plant)_TeamA': 'Time Expiry (Failed to Plant)_TeamB',
        'Elimination_TeamB': 'Elimination_TeamA',
        'Detonated_TeamB': 'Detonated_TeamA',
        'Defused_TeamB': 'Defused_TeamA',
        'Time Expiry (No Plant)_TeamB': 'Time Expiry (No Plant)_TeamA',
        'Eliminated_TeamB': 'Eliminated_TeamA',
        'Defused Failed_TeamB': 'Defused Failed_TeamA',
        'Detonation Denied_TeamB': 'Detonation Denied_TeamA',
        'Time Expiry (Failed to Plant)_TeamB': 'Time Expiry (Failed to Plant)_TeamA'
    })

    # Flip the winner column
    df_flipped['Winner'] = df_flipped['Winner'].apply(lambda x: 1 if x == 0 else 0)

    original_order = [
        'Tournament',
        'Stage',
        'Match Type',
        'Match Name',
        'Map',
        'Team A',
        'Team A Score',
        'Team A Attacker Score',
        'Team A Defender Score',
        'Team A Overtime Score',
        'Team B',
        'Team B Score',
        'Team B Attacker Score',
        'Team B Defender Score',
        'Team B Overtime Score',
        'Duration',
        'Rating_TeamA',
        'Average Combat Score_TeamA',
        'Kills_TeamA',
        'Deaths_TeamA',
        'Assists_TeamA',
        'Kills - Deaths (KD)_TeamA',
        'Kill, Assist, Trade, Survive %_TeamA',
        'Average Damage Per Round_TeamA',
        'Headshot %_TeamA',
        'First Kills_TeamA',
        'First Deaths_TeamA',
        'Kills - Deaths (FKD)_TeamA',
        'Rating_TeamB',
        'Average Combat Score_TeamB',
        'Kills_TeamB',
        'Deaths_TeamB',
        'Assists_TeamB',
        'Kills - Deaths (KD)_TeamB',
        'Kill, Assist, Trade, Survive %_TeamB',
        'Average Damage Per Round_TeamB',
        'Headshot %_TeamB',
        'First Kills_TeamB',
        'First Deaths_TeamB',
        'Kills - Deaths (FKD)_TeamB',
        'Loadout Value_TeamA',
        'Remaining Credits_TeamA',
        'Type_TeamA',
        'Loadout Value_TeamB',
        'Remaining Credits_TeamB',
        'Type_TeamB',
        '2k_TeamA',
        '3k_TeamA',
        '4k_TeamA',
        '5k_TeamA',
        '1v1_TeamA',
        '1v2_TeamA',
        '1v3_TeamA',
        '1v4_TeamA',
        '1v5_TeamA',
        'Econ_TeamA',
        'Spike Plants_TeamA',
        'Spike Defuses_TeamA',
        '2k_TeamB',
        '3k_TeamB',
        '4k_TeamB',
        '5k_TeamB',
        '1v1_TeamB',
        '1v2_TeamB',
        '1v3_TeamB',
        '1v4_TeamB',
        '1v5_TeamB',
        'Econ_TeamB',
        'Spike Plants_TeamB',
        'Spike Defuses_TeamB',
        'Elimination_TeamA',
        'Detonated_TeamA',
        'Defused_TeamA',
        'Time Expiry (No Plant)_TeamA',
        'Eliminated_TeamA',
        'Defused Failed_TeamA',
        'Detonation Denied_TeamA',
        'Time Expiry (Failed to Plant)_TeamA',
        'Elimination_TeamB',
        'Detonated_TeamB',
        'Defused_TeamB',
        'Time Expiry (No Plant)_TeamB',
        'Eliminated_TeamB',
        'Defused Failed_TeamB',
        'Detonation Denied_TeamB',
        'Time Expiry (Failed to Plant)_TeamB',
        'Winner'
    ]


    df_flipped = df_flipped[original_order]

    return df_flipped


# Apply to all years
vct_2021_flipped = flip_teams(vct_2021)
vct_2022_flipped = flip_teams(vct_2022)
vct_2023_flipped = flip_teams(vct_2023)
vct_2024_flipped = flip_teams(vct_2024)

Checking if the flip worked

In [9]:
vct_2024.tail()

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team A Score,Team A Attacker Score,Team A Defender Score,Team A Overtime Score,Team B,Team B Score,Team B Attacker Score,Team B Defender Score,Team B Overtime Score,Duration,Rating_TeamA,Average Combat Score_TeamA,Kills_TeamA,Deaths_TeamA,Assists_TeamA,Kills - Deaths (KD)_TeamA,"Kill, Assist, Trade, Survive %_TeamA",Average Damage Per Round_TeamA,Headshot %_TeamA,First Kills_TeamA,First Deaths_TeamA,Kills - Deaths (FKD)_TeamA,Rating_TeamB,Average Combat Score_TeamB,Kills_TeamB,Deaths_TeamB,Assists_TeamB,Kills - Deaths (KD)_TeamB,"Kill, Assist, Trade, Survive %_TeamB",Average Damage Per Round_TeamB,Headshot %_TeamB,First Kills_TeamB,First Deaths_TeamB,Kills - Deaths (FKD)_TeamB,Loadout Value_TeamA,Remaining Credits_TeamA,Type_TeamA,Loadout Value_TeamB,Remaining Credits_TeamB,Type_TeamB,2k_TeamA,3k_TeamA,4k_TeamA,5k_TeamA,1v1_TeamA,1v2_TeamA,1v3_TeamA,1v4_TeamA,1v5_TeamA,Econ_TeamA,Spike Plants_TeamA,Spike Defuses_TeamA,2k_TeamB,3k_TeamB,4k_TeamB,5k_TeamB,1v1_TeamB,1v2_TeamB,1v3_TeamB,1v4_TeamB,1v5_TeamB,Econ_TeamB,Spike Plants_TeamB,Spike Defuses_TeamB,Elimination_TeamA,Detonated_TeamA,Defused_TeamA,Time Expiry (No Plant)_TeamA,Eliminated_TeamA,Defused Failed_TeamA,Detonation Denied_TeamA,Time Expiry (Failed to Plant)_TeamA,Elimination_TeamB,Detonated_TeamB,Defused_TeamB,Time Expiry (No Plant)_TeamB,Eliminated_TeamB,Defused Failed_TeamB,Detonation Denied_TeamB,Time Expiry (Failed to Plant)_TeamB,Winner
1099,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Haven,EDward Gaming,6,3,3,NaN,Team Heretics,13,4,9,NaN,1:16:43,0.814,182.0,59.0,73.0,21.0,-14.0,0.672,118.0,0.272,8.0,11.0,-3.0,1.208,219.8,73.0,59.0,38.0,14.0,0.802,146.2,0.368,11.0,8.0,3.0,17389.473684,6926.315789,Full buy: 20k+,18194.736842,9878.947368,Full buy: 20k+,11.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,43.8,9.0,0.0,19.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,62.6,3.0,3.0,5,1,0,0,7,0,3,3,7,0,3,3,5,1,0,0,0
1100,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Sunset,EDward Gaming,13,11,2,NaN,Team Heretics,4,3,1,NaN,45:11,1.364,225.6,70.0,40.0,37.0,30.0,0.868,144.8,0.370,12.0,5.0,7.0,0.642,142.0,40.0,70.0,12.0,-30.0,0.542,97.0,0.306,5.0,12.0,-7.0,18494.117647,18241.176471,Full buy: 20k+,15641.176471,4694.117647,Full buy: 20k+,14.0,0.0,3.0,1.0,0.0,2.0,0.0,0.0,0.0,74.2,12.0,1.0,9.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,41.8,4.0,1.0,9,3,1,0,3,0,1,0,3,0,1,0,9,3,1,0,1
1101,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Lotus,EDward Gaming,13,6,7,NaN,Team Heretics,9,3,6,NaN,55:30,1.170,228.4,87.0,72.0,47.0,15.0,0.774,143.4,0.320,11.0,11.0,0.0,0.836,184.4,71.0,87.0,26.0,-16.0,0.608,118.0,0.366,11.0,11.0,0.0,18436.363636,8027.272727,Full buy: 20k+,16540.909091,5600.000000,Full buy: 20k+,13.0,9.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,58.6,8.0,2.0,13.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,45.8,8.0,1.0,10,0,2,1,7,1,1,0,7,1,1,0,10,0,2,1,1
1102,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Bind,EDward Gaming,11,6,5,NaN,Team Heretics,13,7,6,NaN,1:01:45,0.976,203.2,86.0,91.0,32.0,-5.0,0.782,139.8,0.304,10.0,14.0,-4.0,1.038,210.0,91.0,86.0,40.0,5.0,0.818,129.4,0.302,14.0,10.0,4.0,17150.000000,5462.500000,Full buy: 20k+,18404.166667,7033.333333,Full buy: 20k+,14.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,55.2,10.0,3.0,18.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,47.8,10.0,4.0,7,1,3,0,7,1,4,1,7,1,4,1,7,1,3,0,0
1103,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Abyss,EDward Gaming,13,8,5,NaN,Team Heretics,9,5,4,NaN,51:55,1.194,208.6,84.0,63.0,36.0,21.0,0.800,140.0,0.310,13.0,9.0,4.0,0.772,175.4,63.0,88.0,25.0,-25.0,0.692,115.8,0.358,9.0,13.0,-4.0,18486.363636,12090.909091,Full buy: 20k+,15363.636364,8759.090909,Full buy: 20k+,14.0,5.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,61.8,12.0,2.0,11.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,48.0,7.0,2.0,11,0,2,0,2,3,2,2,2,3,2,2,11,0,2,0,1


In [10]:
vct_2024_flipped.tail()

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team A Score,Team A Attacker Score,Team A Defender Score,Team A Overtime Score,Team B,Team B Score,Team B Attacker Score,Team B Defender Score,Team B Overtime Score,Duration,Rating_TeamA,Average Combat Score_TeamA,Kills_TeamA,Deaths_TeamA,Assists_TeamA,Kills - Deaths (KD)_TeamA,"Kill, Assist, Trade, Survive %_TeamA",Average Damage Per Round_TeamA,Headshot %_TeamA,First Kills_TeamA,First Deaths_TeamA,Kills - Deaths (FKD)_TeamA,Rating_TeamB,Average Combat Score_TeamB,Kills_TeamB,Deaths_TeamB,Assists_TeamB,Kills - Deaths (KD)_TeamB,"Kill, Assist, Trade, Survive %_TeamB",Average Damage Per Round_TeamB,Headshot %_TeamB,First Kills_TeamB,First Deaths_TeamB,Kills - Deaths (FKD)_TeamB,Loadout Value_TeamA,Remaining Credits_TeamA,Type_TeamA,Loadout Value_TeamB,Remaining Credits_TeamB,Type_TeamB,2k_TeamA,3k_TeamA,4k_TeamA,5k_TeamA,1v1_TeamA,1v2_TeamA,1v3_TeamA,1v4_TeamA,1v5_TeamA,Econ_TeamA,Spike Plants_TeamA,Spike Defuses_TeamA,2k_TeamB,3k_TeamB,4k_TeamB,5k_TeamB,1v1_TeamB,1v2_TeamB,1v3_TeamB,1v4_TeamB,1v5_TeamB,Econ_TeamB,Spike Plants_TeamB,Spike Defuses_TeamB,Elimination_TeamA,Detonated_TeamA,Defused_TeamA,Time Expiry (No Plant)_TeamA,Eliminated_TeamA,Defused Failed_TeamA,Detonation Denied_TeamA,Time Expiry (Failed to Plant)_TeamA,Elimination_TeamB,Detonated_TeamB,Defused_TeamB,Time Expiry (No Plant)_TeamB,Eliminated_TeamB,Defused Failed_TeamB,Detonation Denied_TeamB,Time Expiry (Failed to Plant)_TeamB,Winner
1099,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Haven,Team Heretics,13,4,9,NaN,EDward Gaming,6,3,3,NaN,1:16:43,1.208,219.8,73.0,59.0,38.0,14.0,0.802,146.2,0.368,11.0,8.0,3.0,0.814,182.0,59.0,73.0,21.0,-14.0,0.672,118.0,0.272,8.0,11.0,-3.0,18194.736842,9878.947368,Full buy: 20k+,17389.473684,6926.315789,Full buy: 20k+,19.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,62.6,3.0,3.0,11.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,43.8,9.0,0.0,7,0,3,3,5,1,0,0,5,1,0,0,7,0,3,3,1
1100,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Sunset,Team Heretics,4,3,1,NaN,EDward Gaming,13,11,2,NaN,45:11,0.642,142.0,40.0,70.0,12.0,-30.0,0.542,97.0,0.306,5.0,12.0,-7.0,1.364,225.6,70.0,40.0,37.0,30.0,0.868,144.8,0.370,12.0,5.0,7.0,15641.176471,4694.117647,Full buy: 20k+,18494.117647,18241.176471,Full buy: 20k+,9.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,41.8,4.0,1.0,14.0,0.0,3.0,1.0,0.0,2.0,0.0,0.0,0.0,74.2,12.0,1.0,3,0,1,0,9,3,1,0,9,3,1,0,3,0,1,0,0
1101,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Lotus,Team Heretics,9,3,6,NaN,EDward Gaming,13,6,7,NaN,55:30,0.836,184.4,71.0,87.0,26.0,-16.0,0.608,118.0,0.366,11.0,11.0,0.0,1.170,228.4,87.0,72.0,47.0,15.0,0.774,143.4,0.320,11.0,11.0,0.0,16540.909091,5600.000000,Full buy: 20k+,18436.363636,8027.272727,Full buy: 20k+,13.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,45.8,8.0,1.0,13.0,9.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,58.6,8.0,2.0,7,1,1,0,10,0,2,1,10,0,2,1,7,1,1,0,0
1102,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Bind,Team Heretics,13,7,6,NaN,EDward Gaming,11,6,5,NaN,1:01:45,1.038,210.0,91.0,86.0,40.0,5.0,0.818,129.4,0.302,14.0,10.0,4.0,0.976,203.2,86.0,91.0,32.0,-5.0,0.782,139.8,0.304,10.0,14.0,-4.0,18404.166667,7033.333333,Full buy: 20k+,17150.000000,5462.500000,Full buy: 20k+,18.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,47.8,10.0,4.0,14.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,55.2,10.0,3.0,7,1,4,1,7,1,3,0,7,1,3,0,7,1,4,1,1
1103,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Abyss,Team Heretics,9,5,4,NaN,EDward Gaming,13,8,5,NaN,51:55,0.772,175.4,63.0,88.0,25.0,-25.0,0.692,115.8,0.358,9.0,13.0,-4.0,1.194,208.6,84.0,63.0,36.0,21.0,0.800,140.0,0.310,13.0,9.0,4.0,15363.636364,8759.090909,Full buy: 20k+,18486.363636,12090.909091,Full buy: 20k+,11.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,48.0,7.0,2.0,14.0,5.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,61.8,12.0,2.0,2,3,2,2,11,0,2,0,11,0,2,0,2,3,2,2,0


# Combine the two variants together

In [11]:
def alternate_concat(df, df_flipped):
    if len(df) != len(df_flipped):
        raise ValueError("Both DataFrames must have the same number of rows")
    
    # Convert to NumPy arrays for faster processing
    arr1 = df.to_numpy()
    arr2 = df_flipped.to_numpy()
    
    # Initialize result array with the correct shape
    result = np.empty((len(df) * 2, df.shape[1]), dtype=arr1.dtype)
    
    # Even indices (0, 2, 4, ...) get rows from df
    result[0::2] = arr1

    # Odd indices (1, 3, 5, ...) get rows from df_flipped
    result[1::2] = arr2
    
    # Convert back to DataFrame
    return pd.DataFrame(result, columns=df.columns)

# Concatenate the original and flipped DataFrames for each year
vct_2021_combined = alternate_concat(vct_2021, vct_2021_flipped)
vct_2022_combined = alternate_concat(vct_2022, vct_2022_flipped)
vct_2023_combined = alternate_concat(vct_2023, vct_2023_flipped)
vct_2024_combined = alternate_concat(vct_2024, vct_2024_flipped)

In [12]:
vct_2024_combined.tail()

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team A Score,Team A Attacker Score,Team A Defender Score,Team A Overtime Score,Team B,Team B Score,Team B Attacker Score,Team B Defender Score,Team B Overtime Score,Duration,Rating_TeamA,Average Combat Score_TeamA,Kills_TeamA,Deaths_TeamA,Assists_TeamA,Kills - Deaths (KD)_TeamA,"Kill, Assist, Trade, Survive %_TeamA",Average Damage Per Round_TeamA,Headshot %_TeamA,First Kills_TeamA,First Deaths_TeamA,Kills - Deaths (FKD)_TeamA,Rating_TeamB,Average Combat Score_TeamB,Kills_TeamB,Deaths_TeamB,Assists_TeamB,Kills - Deaths (KD)_TeamB,"Kill, Assist, Trade, Survive %_TeamB",Average Damage Per Round_TeamB,Headshot %_TeamB,First Kills_TeamB,First Deaths_TeamB,Kills - Deaths (FKD)_TeamB,Loadout Value_TeamA,Remaining Credits_TeamA,Type_TeamA,Loadout Value_TeamB,Remaining Credits_TeamB,Type_TeamB,2k_TeamA,3k_TeamA,4k_TeamA,5k_TeamA,1v1_TeamA,1v2_TeamA,1v3_TeamA,1v4_TeamA,1v5_TeamA,Econ_TeamA,Spike Plants_TeamA,Spike Defuses_TeamA,2k_TeamB,3k_TeamB,4k_TeamB,5k_TeamB,1v1_TeamB,1v2_TeamB,1v3_TeamB,1v4_TeamB,1v5_TeamB,Econ_TeamB,Spike Plants_TeamB,Spike Defuses_TeamB,Elimination_TeamA,Detonated_TeamA,Defused_TeamA,Time Expiry (No Plant)_TeamA,Eliminated_TeamA,Defused Failed_TeamA,Detonation Denied_TeamA,Time Expiry (Failed to Plant)_TeamA,Elimination_TeamB,Detonated_TeamB,Defused_TeamB,Time Expiry (No Plant)_TeamB,Eliminated_TeamB,Defused Failed_TeamB,Detonation Denied_TeamB,Time Expiry (Failed to Plant)_TeamB,Winner
2203,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Lotus,Team Heretics,9,3,6,NaN,EDward Gaming,13,6,7,NaN,55:30,0.836,184.4,71.0,87.0,26.0,-16.0,0.608,118.0,0.366,11.0,11.0,0.0,1.17,228.4,87.0,72.0,47.0,15.0,0.774,143.4,0.32,11.0,11.0,0.0,16540.909091,5600.0,Full buy: 20k+,18436.363636,8027.272727,Full buy: 20k+,13.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,45.8,8.0,1.0,13.0,9.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,58.6,8.0,2.0,7,1,1,0,10,0,2,1,10,0,2,1,7,1,1,0,0
2204,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Bind,EDward Gaming,11,6,5,NaN,Team Heretics,13,7,6,NaN,1:01:45,0.976,203.2,86.0,91.0,32.0,-5.0,0.782,139.8,0.304,10.0,14.0,-4.0,1.038,210.0,91.0,86.0,40.0,5.0,0.818,129.4,0.302,14.0,10.0,4.0,17150.0,5462.5,Full buy: 20k+,18404.166667,7033.333333,Full buy: 20k+,14.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,55.2,10.0,3.0,18.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,47.8,10.0,4.0,7,1,3,0,7,1,4,1,7,1,4,1,7,1,3,0,0
2205,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Bind,Team Heretics,13,7,6,NaN,EDward Gaming,11,6,5,NaN,1:01:45,1.038,210.0,91.0,86.0,40.0,5.0,0.818,129.4,0.302,14.0,10.0,4.0,0.976,203.2,86.0,91.0,32.0,-5.0,0.782,139.8,0.304,10.0,14.0,-4.0,18404.166667,7033.333333,Full buy: 20k+,17150.0,5462.5,Full buy: 20k+,18.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,47.8,10.0,4.0,14.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,55.2,10.0,3.0,7,1,4,1,7,1,3,0,7,1,3,0,7,1,4,1,1
2206,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Abyss,EDward Gaming,13,8,5,NaN,Team Heretics,9,5,4,NaN,51:55,1.194,208.6,84.0,63.0,36.0,21.0,0.8,140.0,0.31,13.0,9.0,4.0,0.772,175.4,63.0,88.0,25.0,-25.0,0.692,115.8,0.358,9.0,13.0,-4.0,18486.363636,12090.909091,Full buy: 20k+,15363.636364,8759.090909,Full buy: 20k+,14.0,5.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,61.8,12.0,2.0,11.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,48.0,7.0,2.0,11,0,2,0,2,3,2,2,2,3,2,2,11,0,2,0,1
2207,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Abyss,Team Heretics,9,5,4,NaN,EDward Gaming,13,8,5,NaN,51:55,0.772,175.4,63.0,88.0,25.0,-25.0,0.692,115.8,0.358,9.0,13.0,-4.0,1.194,208.6,84.0,63.0,36.0,21.0,0.8,140.0,0.31,13.0,9.0,4.0,15363.636364,8759.090909,Full buy: 20k+,18486.363636,12090.909091,Full buy: 20k+,11.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,48.0,7.0,2.0,14.0,5.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,61.8,12.0,2.0,2,3,2,2,11,0,2,0,11,0,2,0,2,3,2,2,0


# Combine all years and save raw dataset for preprocessing later

In [13]:
# Allocate 2021-2023 data for training and 2024 data for testing
# train_data = pd.concat([vct_2021_combined, vct_2022_combined, vct_2023_combined], ignore_index=True)
# test_data = vct_2024_combined

# Combine all years and save resulting dataset into csv
vct_all_raw = pd.concat([vct_2021_combined, vct_2022_combined, vct_2023_combined, vct_2024_combined], ignore_index=True)
vct_all_raw.to_csv("vct_data/vct_all_raw.csv", index=False)